<a href="https://colab.research.google.com/github/Oluwadamilola289/Machine_Learning_diabeties/blob/main/CW_BIG_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cleaning and Preprocessing of Data

In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=b8448aabfdacc166b5d6bce527c1ad262d683beeee32052814009547cc7038d7
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Turkey_developers").getOrCreate()

Load the dataset into PySpark DataFrame:


In [4]:
df = spark.read.csv("/content/salary.csv",
                    header = True, 
                    inferSchema=True)

In [5]:
df.show(1000)

+--------------------+--------------------+--------+----------+--------------------+--------+----------------+----------------+--------------------+--------------------+
|                Time|            Position|   Level|Experience|          Technology|Location|  Way_of_working|Employees_number|         Salary_type|              Salary|
+--------------------+--------------------+--------+----------+--------------------+--------+----------------+----------------+--------------------+--------------------+
|2023-01-22 18:49:...|   Backend Developer|     Mid|   1-3 Yıl|       Java / Spring| Türkiye|Yerinde / Ofiste|         100-300|         Türk Lirası|  41.000 - 45.000 TL|
|2023-01-22 18:49:...|   Architect / Mimar|Tanımsız|   4-6 Yıl|React / NextJS, N...| Türkiye|          Remote|           2000+|         Türk Lirası|  51.000 - 55.000 TL|
|2023-01-22 18:49:...|  Frontend Developer|  Junior|   1-3 Yıl|React / NextJS, J...| Türkiye|          Remote|         100-300|         Türk Lirası|  

In [6]:
print("Number of rows: ", df.count())
print("Number of columns: ", len(df.columns))


Number of rows:  1164
Number of columns:  10


DATA CLEANING

In [7]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import col

df = df.withColumn("Experience", regexp_replace(col("Experience"), "Yıl", ""))


In [8]:
df.show()

+--------------------+--------------------+--------+----------+--------------------+--------+----------------+----------------+-----------+-------------------+
|                Time|            Position|   Level|Experience|          Technology|Location|  Way_of_working|Employees_number|Salary_type|             Salary|
+--------------------+--------------------+--------+----------+--------------------+--------+----------------+----------------+-----------+-------------------+
|2023-01-22 18:49:...|   Backend Developer|     Mid|      1-3 |       Java / Spring| Türkiye|Yerinde / Ofiste|         100-300|Türk Lirası| 41.000 - 45.000 TL|
|2023-01-22 18:49:...|   Architect / Mimar|Tanımsız|      4-6 |React / NextJS, N...| Türkiye|          Remote|           2000+|Türk Lirası| 51.000 - 55.000 TL|
|2023-01-22 18:49:...|  Frontend Developer|  Junior|      1-3 |React / NextJS, J...| Türkiye|          Remote|         100-300|Türk Lirası| 16.000 - 20.000 TL|
|2023-01-22 18:50:...|   Backend Develop

In [9]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("Salary", regexp_replace(col("Salary"), "TL", ""))

In [10]:
df.show(1000)

+--------------------+--------------------+--------+----------+--------------------+--------+----------------+----------------+--------------------+------------------+
|                Time|            Position|   Level|Experience|          Technology|Location|  Way_of_working|Employees_number|         Salary_type|            Salary|
+--------------------+--------------------+--------+----------+--------------------+--------+----------------+----------------+--------------------+------------------+
|2023-01-22 18:49:...|   Backend Developer|     Mid|      1-3 |       Java / Spring| Türkiye|Yerinde / Ofiste|         100-300|         Türk Lirası|  41.000 - 45.000 |
|2023-01-22 18:49:...|   Architect / Mimar|Tanımsız|      4-6 |React / NextJS, N...| Türkiye|          Remote|           2000+|         Türk Lirası|  51.000 - 55.000 |
|2023-01-22 18:49:...|  Frontend Developer|  Junior|      1-3 |React / NextJS, J...| Türkiye|          Remote|         100-300|         Türk Lirası|  16.000 - 2

In [11]:
from pyspark.sql.functions import sum

df.agg(*[sum(df[col].isNull().cast("int")).alias(col) for col in df.columns]).show()


+----+--------+-----+----------+----------+--------+--------------+----------------+-----------+------+
|Time|Position|Level|Experience|Technology|Location|Way_of_working|Employees_number|Salary_type|Salary|
+----+--------+-----+----------+----------+--------+--------------+----------------+-----------+------+
|   0|       0|    0|         0|         0|       0|             0|               0|          0|     0|
+----+--------+-----+----------+----------+--------+--------------+----------------+-----------+------+



In [12]:
df.describe().show()

+-------+--------------------+--------+----------+----------+--------+----------------+----------------+------------------+------------------+
|summary|            Position|   Level|Experience|Technology|Location|  Way_of_working|Employees_number|       Salary_type|            Salary|
+-------+--------------------+--------+----------+----------+--------+----------------+----------------+------------------+------------------+
|  count|                1164|    1164|      1164|      1164|    1164|            1164|            1164|              1164|              1164|
|   mean|                null|    null|      null|      null|    null|            null|            null|            8450.0|              null|
| stddev|                null|    null|      null|      null|    null|            null|            null|10161.815782624677|              null|
|    min|      ABAP Developer|  Junior|      0-1 |         -| Türkiye|          Hybrid|            0-10|             11500|101.000 - 120.000 |

In [13]:
from pyspark.sql.functions import *

# Calculate the counts for each 'Salary'
Salary_count = df.groupBy("Salary").agg(count("*").alias("Salary_count"))

# Define the ranges and create a new column 'Salary_group'
Salary_count = Salary_count.selectExpr(
    "Salary",
    "CASE WHEN Salary_count >= 100.000 THEN '1' \
          WHEN Salary_count >= 50.000 AND Salary_count < 100.000 THEN '2' \
          WHEN Salary_count < 50.000 THEN '3' \
          ELSE '4' END AS Salary_group"
)

# Show the result
Salary_count.show()


+------------------+------------+
|            Salary|Salary_group|
+------------------+------------+
|  21.000 - 25.000 |           1|
|  16.000 - 20.000 |           1|
|  36.000 - 40.000 |           3|
|  66.000 - 70.000 |           3|
|  41.000 - 45.000 |           2|
|  46.000 - 50.000 |           3|
|  81.000 - 90.000 |           3|
|  11.000 - 15.000 |           1|
|151.000 - 200.000 |           3|
|  26.000 - 30.000 |           1|
| 91.000 - 100.000 |           3|
|121.000 - 150.000 |           3|
|         200.000+ |           3|
|    5000 - 10.000 |           1|
|  56.000 - 60.000 |           3|
|  31.000 - 35.000 |           2|
|101.000 - 120.000 |           3|
|  51.000 - 55.000 |           3|
|  61.000 - 65.000 |           3|
|  71.000 - 80.000 |           3|
+------------------+------------+



In [14]:
Developer_Salary = df.join(Salary_count, "Salary", "Left")

In [15]:
Developer_Salary.show()

+-----------------+--------------------+--------------------+--------+----------+--------------------+--------+----------------+----------------+-----------+------------+
|           Salary|                Time|            Position|   Level|Experience|          Technology|Location|  Way_of_working|Employees_number|Salary_type|Salary_group|
+-----------------+--------------------+--------------------+--------+----------+--------------------+--------+----------------+----------------+-----------+------------+
| 41.000 - 45.000 |2023-01-22 18:49:...|   Backend Developer|     Mid|      1-3 |       Java / Spring| Türkiye|Yerinde / Ofiste|         100-300|Türk Lirası|           2|
| 51.000 - 55.000 |2023-01-22 18:49:...|   Architect / Mimar|Tanımsız|      4-6 |React / NextJS, N...| Türkiye|          Remote|           2000+|Türk Lirası|           3|
| 16.000 - 20.000 |2023-01-22 18:49:...|  Frontend Developer|  Junior|      1-3 |React / NextJS, J...| Türkiye|          Remote|         100-300|

In [16]:
from pyspark.sql.functions import *

# Calculate the counts for each 'Experience'
Experience_count = Developer_Salary.groupBy("Experience").agg(count("*").alias("Experience_count"))

# Define the ranges and create a new column 'Experience_group'
exprs = [
    "CASE WHEN Experience = '0-1 ' THEN '1' \
          WHEN Experience = '1-3 ' THEN '2' \
          WHEN Experience = '4-6 ' THEN '3' \
          WHEN Experience = '7-9 ' THEN '4' \
          WHEN Experience = '10+ ' THEN '5' \
          ELSE '0' END AS Experience_group"
]
expr = " ".join(exprs)

# Apply the expression to the DataFrame
Experience_count = Experience_count.selectExpr("Experience", expr)

# Show the result
Experience_count.show()


+----------+----------------+
|Experience|Experience_group|
+----------+----------------+
|      0-1 |               1|
|      4-6 |               3|
|      10+ |               5|
|      1-3 |               2|
|      7-9 |               4|
+----------+----------------+



In [17]:
Developer_Salary = Developer_Salary.join(Experience_count, "Experience", "Left")

In [18]:
Developer_Salary.show()

+----------+-----------------+--------------------+--------------------+--------+--------------------+--------+----------------+----------------+-----------+------------+----------------+
|Experience|           Salary|                Time|            Position|   Level|          Technology|Location|  Way_of_working|Employees_number|Salary_type|Salary_group|Experience_group|
+----------+-----------------+--------------------+--------------------+--------+--------------------+--------+----------------+----------------+-----------+------------+----------------+
|      1-3 | 41.000 - 45.000 |2023-01-22 18:49:...|   Backend Developer|     Mid|       Java / Spring| Türkiye|Yerinde / Ofiste|         100-300|Türk Lirası|           2|               2|
|      4-6 | 51.000 - 55.000 |2023-01-22 18:49:...|   Architect / Mimar|Tanımsız|React / NextJS, N...| Türkiye|          Remote|           2000+|Türk Lirası|           3|               3|
|      1-3 | 16.000 - 20.000 |2023-01-22 18:49:...|  Fronten

In [19]:
Developer_Salary = Developer_Salary.withColumn('clean_column', regexp_replace('Technology', '[^a-zA-Z0-9\\s]', ''))

In [20]:
Developer_Salary=Developer_Salary.drop("Technology")

In [21]:
Developer_Salary.show()

+----------+-----------------+--------------------+--------------------+--------+--------+----------------+----------------+-----------+------------+----------------+--------------------+
|Experience|           Salary|                Time|            Position|   Level|Location|  Way_of_working|Employees_number|Salary_type|Salary_group|Experience_group|        clean_column|
+----------+-----------------+--------------------+--------------------+--------+--------+----------------+----------------+-----------+------------+----------------+--------------------+
|      1-3 | 41.000 - 45.000 |2023-01-22 18:49:...|   Backend Developer|     Mid| Türkiye|Yerinde / Ofiste|         100-300|Türk Lirası|           2|               2|        Java  Spring|
|      4-6 | 51.000 - 55.000 |2023-01-22 18:49:...|   Architect / Mimar|Tanımsız| Türkiye|          Remote|           2000+|Türk Lirası|           3|               3|React  NextJS Nod...|
|      1-3 | 16.000 - 20.000 |2023-01-22 18:49:...|  Fronten

In [22]:
# Assume 'Developer_Salary' is a DataFrame with a column 'old_column'
Developer_Salary = Developer_Salary.withColumnRenamed("clean_column", "Technology")

In [23]:
Developer_Salary.show()

+----------+-----------------+--------------------+--------------------+--------+--------+----------------+----------------+-----------+------------+----------------+--------------------+
|Experience|           Salary|                Time|            Position|   Level|Location|  Way_of_working|Employees_number|Salary_type|Salary_group|Experience_group|          Technology|
+----------+-----------------+--------------------+--------------------+--------+--------+----------------+----------------+-----------+------------+----------------+--------------------+
|      1-3 | 41.000 - 45.000 |2023-01-22 18:49:...|   Backend Developer|     Mid| Türkiye|Yerinde / Ofiste|         100-300|Türk Lirası|           2|               2|        Java  Spring|
|      4-6 | 51.000 - 55.000 |2023-01-22 18:49:...|   Architect / Mimar|Tanımsız| Türkiye|          Remote|           2000+|Türk Lirası|           3|               3|React  NextJS Nod...|
|      1-3 | 16.000 - 20.000 |2023-01-22 18:49:...|  Fronten

So it appers that my dataset was compiled using the Turkish language. I am going to find a way to change those values in to English


In [24]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql import SparkSession

New_df= Developer_Salary.withColumn('Level', regexp_replace('Level', 'Tanımsız', 'Undefined'))\
.withColumn('Salary_type', regexp_replace('Salary_type', 'Dolar', 'Dollar'))\
.withColumn('Way_of_working', regexp_replace('Way_of_working', 'Yerinde / Ofiste', 'Onsite / In Office'))\
.withColumn('Position', regexp_replace('Position', 'Yönetici / Müdür','Executive Director'))\
.withColumn('Position', regexp_replace('Position', 'Game Developer / Oyun Geliştirici','Game Developer'))\
.withColumn('Position', regexp_replace('Position', 'Tasarımcı','Designer'))\
.withColumn('Position', regexp_replace('Position', 'Sap sistem uzmanı','SAP system specialist'))\
.withColumn('Position', regexp_replace('Position', 'Test otomasyon','Test automation'))\
.withColumn('Position', regexp_replace('Position', 'Test mühendisi','Test engineer'))\
.withColumn('Position', regexp_replace('Position', 'Dr. Öğr. Üyesi','Dr. Instructor Member'))\
.withColumn('Position', regexp_replace('Position', 'BT Müfettişi', 'IT Inspector'))\
.withColumn('Position', regexp_replace('Position', 'Koordinatör', 'Coordinator'))\
.withColumn('Position', regexp_replace('Position', 'Siber güvenlik uzmanı', 'Cyber ​​security'))\
.withColumn('Position', regexp_replace('Position', 'Takim lideri', 'Such leaders'))\
.withColumn('Position', regexp_replace('Position', 'Mühendislik analiz', 'Engineering analysis'))\
.withColumn('Position', regexp_replace('Position', 'Makine mühendisi', 'Mechanical engineer'))\
.withColumn('Position', regexp_replace('Position', 'Avukat', 'Lawyer'))\
.withColumn('Position', regexp_replace('Position', 'ERP Yazılımcı', 'ERP Developer'))\
.withColumn('Position', regexp_replace('Position', 'Dwh bi developer', 'Web Developer'))\
.withColumn('Position', regexp_replace('Position', 'Test Otomasyonu', 'Test Automation'))\
.withColumn('Position', regexp_replace('Position', 'Takım lideri', 'Team leader'))\
.withColumn('Position', regexp_replace('Position', 'Siber Güvenlik', 'Cyber ​​security'))\
.withColumn('Experience', regexp_replace('Experience', 'Yıl', 'Years'))\
.withColumn('Salary_type', regexp_replace('Salary_type', 'Sterlin', 'Sterling'))
New_df.show(truncate=False)
 

+----------+-----------------+-----------------------+---------------------+---------+--------+------------------+----------------+-----------+------------+----------------+---------------------------------------------------------------------------------------------------+
|Experience|Salary           |Time                   |Position             |Level    |Location|Way_of_working    |Employees_number|Salary_type|Salary_group|Experience_group|Technology                                                                                         |
+----------+-----------------+-----------------------+---------------------+---------+--------+------------------+----------------+-----------+------------+----------------+---------------------------------------------------------------------------------------------------+
|1-3       |41.000 - 45.000  |2023-01-22 18:49:36.202|Backend Developer    |Mid      |Türkiye |Onsite / In Office|100-300         |Türk Lirası|2           |2               |Java 

In [25]:
New_df.printSchema()
New_df.show(1000)

root
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Position: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Way_of_working: string (nullable = true)
 |-- Employees_number: string (nullable = true)
 |-- Salary_type: string (nullable = true)
 |-- Salary_group: string (nullable = true)
 |-- Experience_group: string (nullable = true)
 |-- Technology: string (nullable = true)

+----------+------------------+--------------------+--------------------+---------+--------+------------------+----------------+--------------------+------------+----------------+--------------------+
|Experience|            Salary|                Time|            Position|    Level|Location|    Way_of_working|Employees_number|         Salary_type|Salary_group|Experience_group|          Technology|
+----------+------------------+--------------------+--------------------+---------+--

In [26]:
filtered_Salary = New_df.where(col("Position").isin ('Backend Developer', 'ABAP Developer','C++ developer ','ERP Developer','CRM Developer','Web Developer','Embedded Software Developer','Frontend Developer','Fullstack Developer','Game Developer','iOS Developer','Mobile Developer','Sql developer'))
filtered_Salary.show(1000)


+----------+------------------+--------------------+--------------------+---------+--------+------------------+----------------+--------------------+------------+----------------+--------------------+
|Experience|            Salary|                Time|            Position|    Level|Location|    Way_of_working|Employees_number|         Salary_type|Salary_group|Experience_group|          Technology|
+----------+------------------+--------------------+--------------------+---------+--------+------------------+----------------+--------------------+------------+----------------+--------------------+
|      1-3 |  41.000 - 45.000 |2023-01-22 18:49:...|   Backend Developer|      Mid| Türkiye|Onsite / In Office|         100-300|         Türk Lirası|           2|               2|        Java  Spring|
|      1-3 |  16.000 - 20.000 |2023-01-22 18:49:...|  Frontend Developer|   Junior| Türkiye|            Remote|         100-300|         Türk Lirası|           1|               2|React  NextJS Jav

In [27]:
print("Number of rows: ", filtered_Salary.count())
print("Number of columns: ", len(filtered_Salary.columns))


Number of rows:  894
Number of columns:  12


In [28]:
from pyspark.sql.functions import col

# Create a list of values to count
positions = ['Backend Developer', 'ABAP Developer', 'C++ developer ', 'CRM Developer', 'Web Developer', 
             'Embedded Software Developer', 'Frontend Developer', 'Fullstack Developer', 
             'Game Developer', 'iOS Developer', 'Mobile Developer', 'Sql developer','ERP Developer']

# Initialize a dictionary to store the counts
count_dict = {}

# Loop through the values in the positions list and count the number of occurrences
for position in positions:
    count_value = filtered_Salary.filter(col("Position") == position).count()
    count_dict[position] = count_value

# Print the counts for each position
for position, count in count_dict.items():
    print(f"Number of {position}: {count}")


Number of Backend Developer: 292
Number of ABAP Developer: 1
Number of C++ developer : 1
Number of CRM Developer: 2
Number of Web Developer: 1
Number of Embedded Software Developer: 1
Number of Frontend Developer: 198
Number of Fullstack Developer: 274
Number of Game Developer: 22
Number of iOS Developer: 1
Number of Mobile Developer: 99
Number of Sql developer: 1
Number of ERP Developer: 1


In [29]:
from pyspark.sql.functions import count

num_rows = filtered_Salary.count()
num_distinct_rows = filtered_Salary.dropDuplicates().count()

if num_rows == num_distinct_rows:
    print("No duplicate rows")
else:
    print("Duplicate rows exist")



No duplicate rows


In [30]:
# Check for missing values
filtered_Salary.select([count(when(col(c).isNull(), c)).alias(c) for c in filtered_Salary.columns if c != 'Time']).show()


+----------+------+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+
|Experience|Salary|Position|Level|Location|Way_of_working|Employees_number|Salary_type|Salary_group|Experience_group|Technology|
+----------+------+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+
|         0|     0|       0|    0|       0|             0|               0|          0|           0|               0|         0|
+----------+------+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+



In [31]:
filtered_Salary.printSchema()

root
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Position: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Way_of_working: string (nullable = true)
 |-- Employees_number: string (nullable = true)
 |-- Salary_type: string (nullable = true)
 |-- Salary_group: string (nullable = true)
 |-- Experience_group: string (nullable = true)
 |-- Technology: string (nullable = true)



In [32]:
from pyspark.sql.functions import col

# Cast 'Salary' column to integer
filtered_Salary = filtered_Salary.withColumn('Salary', col('Salary').cast('integer'))

# Cast 'Experience' column to integer
filtered_Salary = filtered_Salary.withColumn('Experience', col('Experience').cast('integer'))

# Cast 'Salary_type' column to integer
filtered_Salary = filtered_Salary.withColumn('Salary_group', col('Salary_group').cast('integer'))

# Cast 'Experience_group' column to integer
filtered_Salary = filtered_Salary.withColumn('Experience_group', col('Experience_group').cast('integer'))


In [33]:
filtered_Salary.printSchema()

root
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Position: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Way_of_working: string (nullable = true)
 |-- Employees_number: string (nullable = true)
 |-- Salary_type: string (nullable = true)
 |-- Salary_group: integer (nullable = true)
 |-- Experience_group: integer (nullable = true)
 |-- Technology: string (nullable = true)



In [34]:
# Check for missing values
filtered_Salary.select([count(when(col(c).isNull(), c)).alias(c) for c in filtered_Salary.columns if c != 'Time']).show()


+----------+------+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+
|Experience|Salary|Position|Level|Location|Way_of_working|Employees_number|Salary_type|Salary_group|Experience_group|Technology|
+----------+------+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+
|       894|   894|       0|    0|       0|             0|               0|          0|           0|               0|         0|
+----------+------+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+



In [35]:
 # Drop missing values
filtered_Salary = filtered_Salary.drop('Experience', 'Salary')

In [36]:
filtered_Salary.select([count(when(col(c).isNull(), c)).alias(c) for c in filtered_Salary.columns if c != 'Time']).show()


+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+
|Position|Level|Location|Way_of_working|Employees_number|Salary_type|Salary_group|Experience_group|Technology|
+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+
|       0|    0|       0|             0|               0|          0|           0|               0|         0|
+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+



In [37]:
print("Number of rows: ", filtered_Salary.count())
print("Number of columns: ", len(filtered_Salary.columns))


Number of rows:  894
Number of columns:  10


In [38]:
# Check for missing values
filt_Sal = filtered_Salary.select([count(when(col(c).isNull(), c)).alias(c) for c in filtered_Salary.columns if c != 'Time']).show()


+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+
|Position|Level|Location|Way_of_working|Employees_number|Salary_type|Salary_group|Experience_group|Technology|
+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+
|       0|    0|       0|             0|               0|          0|           0|               0|         0|
+--------+-----+--------+--------------+----------------+-----------+------------+----------------+----------+



In [39]:
filtered_Salary.describe().show()

+-------+--------------+---------+--------+--------------+----------------+------------------+------------------+------------------+----------+
|summary|      Position|    Level|Location|Way_of_working|Employees_number|       Salary_type|      Salary_group|  Experience_group|Technology|
+-------+--------------+---------+--------+--------------+----------------+------------------+------------------+------------------+----------+
|  count|           894|      894|     894|           894|             894|               894|               894|               894|       894|
|   mean|          null|     null|    null|          null|            null|           10187.5|1.4821029082774049| 2.453020134228188|      null|
| stddev|          null|     null|    null|          null|            null|10842.384039807233|0.7785000411985011|1.2183400882769893|      null|
|    min|ABAP Developer|   Junior| Türkiye|        Hybrid|            0-10|             11500|                 1|                 1|    

In [40]:
N_DS = filtered_Salary

In [41]:
N_DS.describe().show()

+-------+--------------+---------+--------+--------------+----------------+------------------+------------------+------------------+----------+
|summary|      Position|    Level|Location|Way_of_working|Employees_number|       Salary_type|      Salary_group|  Experience_group|Technology|
+-------+--------------+---------+--------+--------------+----------------+------------------+------------------+------------------+----------+
|  count|           894|      894|     894|           894|             894|               894|               894|               894|       894|
|   mean|          null|     null|    null|          null|            null|           10187.5|1.4821029082774049| 2.453020134228188|      null|
| stddev|          null|     null|    null|          null|            null|10842.384039807233|0.7785000411985011|1.2183400882769893|      null|
|    min|ABAP Developer|   Junior| Türkiye|        Hybrid|            0-10|             11500|                 1|                 1|    

In [42]:
N_DS.printSchema()

root
 |-- Time: timestamp (nullable = true)
 |-- Position: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Way_of_working: string (nullable = true)
 |-- Employees_number: string (nullable = true)
 |-- Salary_type: string (nullable = true)
 |-- Salary_group: integer (nullable = true)
 |-- Experience_group: integer (nullable = true)
 |-- Technology: string (nullable = true)



In [43]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

categ_column = ["Position", "Level"]

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in categ_column]

# set the stages from the list of indexers above 
PL = Pipeline(stages=indexers)
mod = PL.fit(N_DS) #model fit to engage the pipeline

In [44]:
N_DS_idx = mod.transform(N_DS) #transforming the dataset to accommodate the model fit

In [45]:
N_DS_idx.printSchema()

root
 |-- Time: timestamp (nullable = true)
 |-- Position: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Way_of_working: string (nullable = true)
 |-- Employees_number: string (nullable = true)
 |-- Salary_type: string (nullable = true)
 |-- Salary_group: integer (nullable = true)
 |-- Experience_group: integer (nullable = true)
 |-- Technology: string (nullable = true)
 |-- Position_index: double (nullable = false)
 |-- Level_index: double (nullable = false)



In [46]:
Salarynew = N_DS_idx[["Position_index", "Level_index", "Experience_group", "Salary_group"]]

In [47]:
Salarynew.show(30)

+--------------+-----------+----------------+------------+
|Position_index|Level_index|Experience_group|Salary_group|
+--------------+-----------+----------------+------------+
|           0.0|        1.0|               2|           2|
|           2.0|        0.0|               2|           1|
|           0.0|        1.0|               3|           2|
|           1.0|        0.0|               1|           1|
|           0.0|        0.0|               1|           1|
|           1.0|        0.0|               2|           1|
|           1.0|        0.0|               1|           1|
|           3.0|        1.0|               3|           2|
|           2.0|        0.0|               2|           1|
|           1.0|        0.0|               1|           1|
|           0.0|        1.0|               2|           1|
|           0.0|        2.0|               4|           3|
|           3.0|        2.0|               3|           2|
|           0.0|        1.0|               2|           

In [48]:
input = Salarynew.columns
input = input[0:-1]
input

['Position_index', 'Level_index', 'Experience_group']

In [49]:
TARGET = "Salary_group"

In [50]:
changecol = Salarynew.withColumn("Salary_group", Salarynew[TARGET])
idx = StringIndexer(inputCol="Salary_group", outputCol="label")
IDX_N_Ds = idx.fit(changecol).transform(changecol)

In [51]:
from pyspark.ml.feature import VectorAssembler
from py4j.java_gateway import select

INPUT = ["Position_index", "Level_index", "Experience_group"]
# Create a feature vector by combining all the feature columns
ASSEMBLE = VectorAssembler(inputCols=INPUT, outputCol="features")

OUTPUT_SALARY = ASSEMBLE.transform(IDX_N_Ds).select("features", "Salary_group")
OUTPUT_SALARY.show()



+-------------+------------+
|     features|Salary_group|
+-------------+------------+
|[0.0,1.0,2.0]|           2|
|[2.0,0.0,2.0]|           1|
|[0.0,1.0,3.0]|           2|
|[1.0,0.0,1.0]|           1|
|[0.0,0.0,1.0]|           1|
|[1.0,0.0,2.0]|           1|
|[1.0,0.0,1.0]|           1|
|[3.0,1.0,3.0]|           2|
|[2.0,0.0,2.0]|           1|
|[1.0,0.0,1.0]|           1|
|[0.0,1.0,2.0]|           1|
|[0.0,2.0,4.0]|           3|
|[3.0,2.0,3.0]|           2|
|[0.0,1.0,2.0]|           1|
|[2.0,0.0,1.0]|           1|
|[0.0,0.0,1.0]|           1|
|[0.0,1.0,2.0]|           1|
|[0.0,1.0,2.0]|           1|
|[2.0,2.0,5.0]|           3|
|[1.0,1.0,2.0]|           1|
+-------------+------------+
only showing top 20 rows



In [52]:
Salary_count = OUTPUT_SALARY.groupBy("Salary_group").count()
Salary_count.show()

+------------+-----+
|Salary_group|count|
+------------+-----+
|           1|  622|
|           3|  159|
|           2|  113|
+------------+-----+



In [53]:
featurelist = INPUT 
# Create a feature vector by combining all the feature columns
ASSEMBLED = VectorAssembler(inputCols=featurelist, outputCol="features")

OUTPUT_SAL = ASSEMBLED.transform(IDX_N_Ds).select("features", "label")

In [54]:
OUTPUT_SAL.show()

+-------------+-----+
|     features|label|
+-------------+-----+
|[0.0,1.0,2.0]|  2.0|
|[2.0,0.0,2.0]|  0.0|
|[0.0,1.0,3.0]|  2.0|
|[1.0,0.0,1.0]|  0.0|
|[0.0,0.0,1.0]|  0.0|
|[1.0,0.0,2.0]|  0.0|
|[1.0,0.0,1.0]|  0.0|
|[3.0,1.0,3.0]|  2.0|
|[2.0,0.0,2.0]|  0.0|
|[1.0,0.0,1.0]|  0.0|
|[0.0,1.0,2.0]|  0.0|
|[0.0,2.0,4.0]|  1.0|
|[3.0,2.0,3.0]|  2.0|
|[0.0,1.0,2.0]|  0.0|
|[2.0,0.0,1.0]|  0.0|
|[0.0,0.0,1.0]|  0.0|
|[0.0,1.0,2.0]|  0.0|
|[0.0,1.0,2.0]|  0.0|
|[2.0,2.0,5.0]|  1.0|
|[1.0,1.0,2.0]|  0.0|
+-------------+-----+
only showing top 20 rows



In [55]:
# Split the data into training and testing sets
(training_data, testing_data) = OUTPUT_SAL.randomSplit([0.8, 0.2])



In [56]:
training_data.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|               705|
|   mean|0.4453900709219858|
| stddev| 0.719204419324766|
|    min|               0.0|
|    max|               2.0|
+-------+------------------+



In [57]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import FloatType

# Train a logistic regression model
lr = LogisticRegression()
lr_model = lr.fit(training_data)
lr_predictions = lr_model.transform(testing_data)




In [58]:
DS_Evaluator= MulticlassClassificationEvaluator(metricName="accuracy") 
accuracy = (DS_Evaluator.evaluate(lr_predictions))*100

DS_Evaluator.setMetricName("weightedPrecision")
precision = DS_Evaluator.evaluate(lr_predictions)

DS_Evaluator.setMetricName("weightedRecall")
recall = DS_Evaluator.evaluate(lr_predictions)

DS_Evaluator.setMetricName("f1")
f1 = DS_Evaluator.evaluate(lr_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 76.71957671957672
Precision: 0.7349814661642617
Recall: 0.7671957671957672
F1-score: 0.7308554121339509


In [59]:
# Train a Randomforest model
rf = RandomForestClassifier()
rf_model = rf.fit(training_data)
rf_predictions = rf_model.transform(testing_data)

DS_Evaluator = MulticlassClassificationEvaluator(metricName="accuracy") 
accuracy = (DS_Evaluator.evaluate(rf_predictions))*100

DS_Evaluator.setMetricName("weightedPrecision")
precision = DS_Evaluator.evaluate(rf_predictions)

DS_Evaluator.setMetricName("weightedRecall")
recall = DS_Evaluator.evaluate(rf_predictions)

DS_Evaluator.setMetricName("f1")
f1 = DS_Evaluator.evaluate(rf_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)



Accuracy: 75.66137566137566
Precision: 0.7139997139997141
Recall: 0.7566137566137565
F1-score: 0.7315373723824428


In [60]:
# Train a decision tree class model
dt = DecisionTreeClassifier()
dt_model = dt.fit(training_data)
dt_predictions = dt_model.transform(testing_data)

# evaluate
DS_Evaluator = MulticlassClassificationEvaluator(metricName="accuracy") 
accuracy = (DS_Evaluator.evaluate(dt_predictions))*100

DS_Evaluator.setMetricName("weightedPrecision")
precision = DS_Evaluator.evaluate(dt_predictions)

DS_Evaluator.setMetricName("weightedRecall")
recall = DS_Evaluator.evaluate(dt_predictions)

DS_Evaluator.setMetricName("f1")
f1 = DS_Evaluator.evaluate(dt_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 75.66137566137566
Precision: 0.7284202569916856
Recall: 0.7566137566137565
F1-score: 0.7371415758867602


In [61]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create a Naive Bayes model
nb = NaiveBayes()

# Fit the model to the training data
nb_model = nb.fit(training_data)

# Make predictions on the testing data
nb_predictions = nb_model.transform(testing_data)

# Evaluate the model's performance
DS_Evaluator = MulticlassClassificationEvaluator(metricName="accuracy") 
accuracy = (DS_Evaluator.evaluate(nb_predictions))*100

DS_Evaluator.setMetricName("weightedPrecision")
precision = DS_Evaluator.evaluate(nb_predictions)

DS_Evaluator.setMetricName("weightedRecall")
recall = DS_Evaluator.evaluate(nb_predictions)

DS_Evaluator.setMetricName("f1")
f1 = DS_Evaluator.evaluate(nb_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 75.13227513227513
Precision: 0.6568102796674224
Recall: 0.7513227513227513
F1-score: 0.6805540365485243


In [62]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Create a Naive Bayes model
nb = NaiveBayes()

# Define the hyperparameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]) \
    .build()

# Define the cross-validation object
crossval = CrossValidator(estimator=nb,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName="accuracy"),
                          numFolds=5)

# Fit the cross-validation object to the training data
cvModel = crossval.fit(training_data)

# Make predictions on the testing data using the best model
nb_predictions = cvModel.transform(testing_data)

# Evaluate the model's performance
DS_Evaluator = MulticlassClassificationEvaluator(metricName="accuracy") 
accuracy = (DS_Evaluator.evaluate(nb_predictions))*100

DS_Evaluator.setMetricName("weightedPrecision")
precision = DS_Evaluator.evaluate(nb_predictions)

DS_Evaluator.setMetricName("weightedRecall")
recall = DS_Evaluator.evaluate(nb_predictions)

DS_Evaluator.setMetricName("f1")
f1 = DS_Evaluator.evaluate(nb_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Print the best hyperparameters
print("Best hyperparameters:", cvModel.bestModel.extractParamMap())


Accuracy: 75.13227513227513
Precision: 0.6568102796674224
Recall: 0.7513227513227513
F1-score: 0.6805540365485243
Best hyperparameters: {Param(parent='NaiveBayes_dc0552a5da89', name='featuresCol', doc='features column name.'): 'features', Param(parent='NaiveBayes_dc0552a5da89', name='labelCol', doc='label column name.'): 'label', Param(parent='NaiveBayes_dc0552a5da89', name='modelType', doc='The model type which is a string (case-sensitive). Supported options: multinomial (default), bernoulli and gaussian.'): 'multinomial', Param(parent='NaiveBayes_dc0552a5da89', name='predictionCol', doc='prediction column name.'): 'prediction', Param(parent='NaiveBayes_dc0552a5da89', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities.'): 'probability', Param(parent='NaiveBayes_dc0552a5da89', name='rawPredictionCol', doc=

In [63]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Create a Decision Tree model
dt = DecisionTreeClassifier()

# Define the hyperparameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15, 20]) \
    .addGrid(dt.minInstancesPerNode, [1, 5, 10, 15]) \
    .build()

# Define the cross-validation object
crossval = CrossValidator(estimator=dt,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName="accuracy"),
                          numFolds=5)

# Fit the cross-validation object to the training data
cvModel = crossval.fit(training_data)

# Make predictions on the testing data using the best model
dt_predictions = cvModel.transform(testing_data)

# Evaluate the model's performance
MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") 
accuracy = (MC_evaluator.evaluate(dt_predictions))*100

MC_evaluator.setMetricName("weightedPrecision")
precision = MC_evaluator.evaluate(dt_predictions)

MC_evaluator.setMetricName("weightedRecall")
recall = MC_evaluator.evaluate(dt_predictions)

MC_evaluator.setMetricName("f1")
f1 = MC_evaluator.evaluate(dt_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Print the best hyperparameters
print("Best hyperparameters:", cvModel.bestModel.extractParamMap())


Accuracy: 75.66137566137566
Precision: 0.7284202569916856
Recall: 0.7566137566137565
F1-score: 0.7371415758867602
Best hyperparameters: {Param(parent='DecisionTreeClassifier_e3dfff347f8a', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False, Param(parent='DecisionTreeClassifier_e3dfff347f8a', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10, Param(parent='DecisionTreeClassifier_e3dfff347f8a', name='featuresCol', doc='features column name.'): 'features', Param(parent='DecisionTreeClassifier_e3dfff347f8a

In [64]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Create a logistic regression model
lr = LogisticRegression()

# Define the hyperparameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

# Define the cross-validation object
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName="accuracy"),
                          numFolds=5)

# Fit the cross-validation object to the training data
cvModel = crossval.fit(training_data)

# Make predictions on the testing data using the best model
lr_predictions = cvModel.transform(testing_data)

# Evaluate the model's performance
MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") 
accuracy = (MC_evaluator.evaluate(lr_predictions))*100

MC_evaluator.setMetricName("weightedPrecision")
precision = MC_evaluator.evaluate(lr_predictions)

MC_evaluator.setMetricName("weightedRecall")
recall = MC_evaluator.evaluate(lr_predictions)

MC_evaluator.setMetricName("f1")
f1 = MC_evaluator.evaluate(lr_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Print the best hyperparameters
print("Best hyperparameters:", cvModel.bestModel.extractParamMap())


Accuracy: 74.60317460317461
Precision: 0.653817082388511
Recall: 0.746031746031746
F1-score: 0.6881787802840434
Best hyperparameters: {Param(parent='LogisticRegression_b67286c0b853', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LogisticRegression_b67286c0b853', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_b67286c0b853', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto', Param(parent='LogisticRegression_b67286c0b853', name='featuresCol', doc='features column name.'): 'features', Param(parent='LogisticRegression_b67286c0b853', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_b67286c0b853', name='labelCol', doc='label c

In [65]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Define the hyperparameter grid
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 50, 100]) \
    .addGrid(rf.maxDepth, [5, 10, 15, 20]) \
    .addGrid(rf.featureSubsetStrategy, ['auto', 'sqrt', 'log2']) \
    .build()

# Define the cross-validation object
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(metricName="accuracy"),
                          numFolds=5)

# Fit the cross-validation object to the training data
cvModel = crossval.fit(training_data)

# Make predictions on the testing data using the best model
rf_predictions = cvModel.transform(testing_data)

# Evaluate the model's performance
MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") 
accuracy = (MC_evaluator.evaluate(rf_predictions))*100

MC_evaluator.setMetricName("weightedPrecision")
precision = MC_evaluator.evaluate(rf_predictions)

MC_evaluator.setMetricName("weightedRecall")
recall = MC_evaluator.evaluate(rf_predictions)

MC_evaluator.setMetricName("f1")
f1 = MC_evaluator.evaluate(rf_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Print the best hyperparameters
print("Best hyperparameters:", cvModel.bestModel.extractParamMap())


Accuracy: 74.60317460317461
Precision: 0.7248474588282333
Recall: 0.746031746031746
F1-score: 0.730968456037808
Best hyperparameters: {Param(parent='RandomForestClassifier_937edb27ff10', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True, Param(parent='RandomForestClassifier_937edb27ff10', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False, Param(parent='RandomForestClassifier_937edb27ff10', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10, Param(parent='RandomFores

In [67]:
# Save the DataFrame as a CSV file
N_DS.write.format("csv").option("header", "true").mode("overwrite").save("path/to/file.csv")